In [15]:
# Import dependencies
import numpy as np
import pandas as pd
import gmaps
import requests
from config import g_key
from config import weather_api_key
import matplotlib.pyplot as plt
import os
from citipy import citipy
import json
from datetime import datetime
import time

In [3]:
# Generate random coordinates
random_lats = np.random.uniform(-90,90,2000)
random_longs = np.random.uniform(-180,180,2000)
lat_longs = zip(random_lats, random_longs)
coordinates = list(lat_longs)

In [9]:
# Create list of cities nearest to random coordinates
cities = []
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0],coordinate[1]).city_name
    if city not in cities:
        cities.append(city)
print(f"Returned {len(cities)} random cities accross the world")


Returned 754 random cities accross the world


In [25]:
# Make dataframe containing weather data for each city in list 'cities'
start_time = time.time()
# Define URL
base_url = 'http://api.openweathermap.org/data/2.5/weather?q='
units = 'Imperial'

# Make variables to count the number of good and bad requests.get() pulls from OpenWeatherMap
cities_found = 0
cities_not_found = 0

weather_list = []

for i in range(0,len(cities)):
    city = cities[i]
    try:
        city_weather = requests.get(f'{base_url}{city}&units={units}&APPID={weather_api_key}').json()
        city_lat = city_weather['coord']['lat']
        city_long = city_weather['coord']['lon']
        city_max_temp = city_weather['main']['temp_max']
        city_humidity = city_weather['main']['humidity']
        city_clouds = city_weather['clouds']['all']
        city_wind = city_weather['wind']['speed']
        city_desc = city_weather['weather'][0]['description']
        weather_list.append({
            'City':city.title(),
            'Country':city_weather['sys']['country'],
            'Lat':city_lat,
            'Long':city_long,
            'Max Temp':city_max_temp,
            'Humidity':city_humidity,
            'Cloudiness':city_clouds,
            'Wind Speed':city_wind,
            'Current Weather':city_desc})
        cities_found += 1
    except KeyError:
        cities_not_found += 1

end_time = time.time()
print(f'City search complete. {cities_found} cities were found, {cities_not_found} cities were not found.\nSearch took {end_time - start_time:.0f} seconds.')

City search complete. 693 cities were found, 61 cities were not found.
Search took 134 seconds.


In [34]:
# Cast list of dictionaries to dataframe

city_data_df = pd.DataFrame(weather_list)

In [35]:
# Reorder columns

city_data_df = city_data_df[['City','Country','Lat','Long','Max Temp','Humidity','Cloudiness','Wind Speed','Current Weather']]
city_data_df.head()

,City,Country,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Current Weather
0,Los Llanos De Aridane,ES,28.6585,-17.9182,66.2,72,75,16.11,broken clouds
1,Ushuaia,AR,-54.8000,-68.3000,51.8,62,20,26.46,few clouds
2,Marin,US,38.0666,-122.7344,50.0,93,90,4.65,overcast clouds
3,Tautira,PF,-17.7333,-149.1500,78.8,69,75,5.75,light rain
4,Mataura,NZ,-46.1927,168.8643,48.0,80,50,8.01,scattered clouds


In [30]:
# Drop any rows with missing data

city_data_df = city_data_df.dropna()

In [36]:
# Write the dataframe to a csv in the current directory

city_data_df.to_csv('./WeatherPy_Database.csv')